In [1]:
!pip install -q openai pinecone python-dotenv pandas

In [2]:
import os, time, json
from typing import Any, Dict, List
import pandas as pd

from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
from pinecone import Pinecone
load_dotenv(find_dotenv(usecwd=True))

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
assert OPENAI_API_KEY and PINECONE_API_KEY, "환경변수(OPENAI_API_KEY, PINECONE_API_KEY)를 .env에 넣어주세요."

INDEX_NAME = "perfume-rag"
NAMESPACE  = "catalog"
EMBED_MODEL = "text-embedding-3-small"
CHAT_MODEL  = "gpt-4o-mini"

oa = OpenAI(api_key=OPENAI_API_KEY)
pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

c:\Users\playdata2\miniconda3\envs\web_server_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from statistics import mean, median
from textwrap import dedent

def embed(text: str) -> List[float]:
    return oa.embeddings.create(model=EMBED_MODEL, input=text).data[0].embedding

def pinecone_search(qvec: List[float], top_k=5, filt: Dict[str, Any] | None = None):
    t0 = time.perf_counter()
    res = index.query(
        vector=qvec, top_k=top_k, include_metadata=True,
        include_values=False, namespace=NAMESPACE,
        filter=filt if filt else None
    )
    t1 = time.perf_counter()
    return res, (t1 - t0) * 1000.0  # ms

def format_context(matches) -> str:
    lines = []
    for i, m in enumerate(matches, 1):
        md = m.metadata or {}
        brand = md.get("brand", "")
        name  = md.get("name", "")
        conc  = md.get("concentration", "")
        price = md.get("price_krw", "")
        size  = md.get("size_ml", "")
        accords = ", ".join(md.get("main_accords", [])[:5])
        top    = ", ".join(md.get("top_notes", [])[:5])
        mid    = ", ".join(md.get("middle_notes", [])[:5])
        base   = ", ".join(md.get("base_notes", [])[:5])
        parts = [f"[{i}] {brand} | {name}"]
        if conc: parts.append(f"농도: {conc}")
        if size: parts.append(f"{size}ml")
        if price: parts.append(f"{price}원")
        if accords: parts.append(f"메인 어코드: {accords}")
        if top: parts.append(f"탑: {top}")
        if mid: parts.append(f"미들: {mid}")
        if base: parts.append(f"베이스: {base}")
        lines.append(" · ".join(parts))
    return "\n".join(lines)

def build_prompt(question: str, context: str) -> List[Dict[str, str]]:
    sys = dedent("""
        너는 향수 추천 어시스턴트야. 아래 컨텍스트만 근거로 간결히 답해.
        모르면 모른다고 말하고, 추측하지 마.
    """).strip()
    user = f"질문: {question}\n\n컨텍스트:\n{context}"
    return [{"role":"system","content":sys}, {"role":"user","content":user}]

def ask_llm(question: str, context: str) -> tuple[str, float]:
    msgs = build_prompt(question, context)
    t0 = time.perf_counter()
    resp = oa.chat.completions.create(model=CHAT_MODEL, messages=msgs, temperature=0.2)
    t1 = time.perf_counter()
    answer = resp.choices[0].message.content
    return answer, (t1 - t0) * 1000.0


In [9]:
def benchmark(question: str, filt: Dict[str, Any] | None, top_k=5, repeats=5):
    qvec = embed(question)
    search_ms_list, llm_ms_list, total_ms_list = [], [], []

    last_context, last_matches = "", None
    for _ in range(repeats):
        res, search_ms = pinecone_search(qvec, top_k=top_k, filt=filt)
        matches = res.matches or []
        context = format_context(matches)
        _, llm_ms = ask_llm(question, context)

        search_ms_list.append(search_ms)
        llm_ms_list.append(llm_ms)
        total_ms_list.append(search_ms + llm_ms)
        last_context, last_matches = context, matches

    summary = {
        "mode": "with_filter" if filt else "no_filter",
        "top_k": top_k,
        "repeats": repeats,
        "검색 평균 속도": round(mean(search_ms_list), 1),
        "검색 중앙값": round(median(search_ms_list), 1),
        "LLM 호출 평균": round(mean(llm_ms_list), 1),
        "검색 + LLM": round(mean(total_ms_list), 1),
    }
    details = {
        "search_times_ms": search_ms_list,
        "llm_times_ms": llm_ms_list,
        "total_times_ms": total_ms_list,
        "last_context": last_context,
        "last_matches": [m.metadata for m in (last_matches or [])],
    }
    return {"summary": summary, "details": details}

In [10]:
question = "여름에 쓰기 좋은 시트러스 향 추천"

res_no = benchmark(question, filt=None, top_k=5, repeats=5)

filt = {"main_accords": {"$in": ["시트러스"]}}
res_f = benchmark(question, filt=filt, top_k=5, repeats=5)

pd.DataFrame([res_no["summary"], res_f["summary"]]).set_index("mode")

,top_k,repeats,검색 평균 속도,검색 중앙값,LLM 호출 평균,검색 + LLM
mode,,,,,,
no_filter,5,5,391.0,217.3,1616.9,2007.9
with_filter,5,5,211.6,212.5,1279.7,1491.3


In [6]:
print("----- no_filter context -----")
print(res_no["details"]["last_context"][:1000], "\n")

print("----- with_filter context -----")
print(res_f["details"]["last_context"][:1000], "\n")

print("개별 검색(ms):", res_no["details"]["search_times_ms"], "→ no_filter")
print("개별 검색(ms):", res_f["details"]["search_times_ms"], "→ with_filter")


----- no_filter context -----
[1] 메종 마르지엘라 | 언더 더 레몬 트리 오 드 뚜왈렛 · 농도: 오 드 뚜왈렛 · 100.0ml · 1514200.0원 · 메인 어코드: 시트러스, 그린, 아로마틱 · 탑: 라임, 페티그레인, 카다멈 · 미들: 그린티, 마테, 코리안더 · 베이스: 화이트 머스크, 시더, 록 로즈
[2] 메종 마르지엘라 | 언더 더 레몬 트리 오 드 뚜왈렛 · 농도: 오 드 뚜왈렛 · 100.0ml · 1514200.0원 · 메인 어코드: 시트러스, 그린, 아로마틱 · 탑: 라임, 페티그레인, 카다멈 · 미들: 그린티, 마테, 코리안더 · 베이스: 화이트 머스크, 시더, 록 로즈
[3] 메종 마르지엘라 | 레이지 선데이 모닝 오 드 뚜왈렛 · 농도: 오 드 뚜왈렛 · 100.0ml · 1396200.0원 · 메인 어코드: 머스크, 프레쉬, 화이트 플로랄 · 탑: 알데하이드, 릴리오브더밸리, 페어 · 미들: 장미, 아이리스, 오렌지 블로썸 · 베이스: 화이트 머스크, 암브레트, 인도네시아 페출리 리프
[4] 메종 마르지엘라 | 레이지 선데이 모닝 오 드 뚜왈렛 · 농도: 오 드 뚜왈렛 · 30.0ml · 750000.0원 · 메인 어코드: 머스크, 프레쉬, 화이트 플로랄 · 탑: 알데하이드, 릴리오브더밸리, 페어 · 미들: 장미, 아이리스, 오렌지 블로썸 · 베이스: 화이트 머스크, 암브레트, 인도네시아 페출리 리프
[5] 메종 마르지엘라 | 레이지 선데이 모닝 오 드 뚜왈렛 · 농도: 오 드 뚜왈렛 · 30.0ml · 750000.0원 · 메인 어코드: 머스크, 프레쉬, 화이트 플로랄 · 탑: 알데하이드, 릴리오브더밸리, 페어 · 미들: 장미, 아이리스, 오렌지 블로썸 · 베이스: 화이트 머스크, 암브레트, 인도네시아 페출리 리프 

----- with_filter context -----
[1] 메종 마르지엘라 | 언더 더 레몬 트리 오 드 뚜왈렛 · 농도: 오 드 뚜왈렛 · 100.0ml · 1514200.0원 ·

In [ ]:
def compliance_rate(matches, cond_key, cond_values):
    ok, total = 0, 0
    for item in matches:
        md = getattr(item, "metadata", item) or {}
        vals = md.get(cond_key, [])
        if not isinstance(vals, list):
            vals = [vals] if vals is not None else []
        if any(v in vals for v in cond_values):
            ok += 1
        total += 1
    return (ok / total) if total else 0.0

rate_no  = compliance_rate(res_no["details"]["last_matches"],  "main_accords", ["시트러스"])
rate_flt = compliance_rate(res_f["details"]["last_matches"],   "main_accords", ["시트러스"])
print("필터 X 조건 만족율:", rate_no, " | 필터 조건 만족율:", rate_flt)


무필터 준수율: 0.4  | 필터 준수율: 1.0
